$$
\max_{\pi_{RL}} \quad \mathbb{E}_{x \sim \mathcal{D}, y \sim \pi_{RL}(\cdot|x)} \left[ r_{\phi}(x, y) - \beta \cdot D_{KL}(\pi_{RL}(\cdot|x) || \pi_{SFT}(\cdot|x)) \right]
$$
- $r_{\phi}(x, y)$：由奖励模型（Reward Model）给出的标量奖励，定义在句子级别

$$
R_t = \begin{cases} 
      -\beta \log\left(\frac{\pi_{RL}(y_t|x, y_{<t})}{\pi_{SFT}(y_t|x, y_{<t})}\right) & \text{if } t < T \\
      r_{\phi}(x, y) - \beta \log\left(\frac{\pi_{RL}(y_t|x, y_{<t})}{\pi_{SFT}(y_t|x, y_{<t})}\right) & \text{if } t = T 
   \end{cases}
$$

- 为了使用像 PPO 这样的 RL 算法，我们需要一个在每个时间步（即每个 token 生成时）都能计算的奖励信号。
- 我们将上述目标函数中的两部分组合成一个单一的、逐 token 的奖励信号。对于一个由 T 个 token 组成的生成序列 $y = (y_1, y_2, ..., y_T)$：
    - 在大多数时间步 t < T，奖励只包含 KL 惩罚项。
    - 在最后一个时间步 T，奖励既包含来自奖励模型的最终分数，也包含该步的 KL 惩罚。
- 通过这种方式，我们将理论目标巧妙地编码成了 PPO 算法可以使用的即时奖励信号。当 PPO 试图最大化这些奖励的累计期望时，它实际上就在优化我们最初设定的 RLHF 目标。

$$
L^{CLIP}(\theta) = \hat{\mathbb{E}}_t \left[ \min\left( \rho_t(\theta) \hat{A}_t, \quad \text{clip}(\rho_t(\theta), 1-\epsilon, 1+\epsilon) \hat{A}_t \right) \right]
$$
- $\rho_t(\theta) = \frac{\pi_{\theta}(a_t|s_t)}{\pi_{\theta_{old}}(a_t|s_t)}$
- $\hat{A}_t = (\sum_{k=t}^{T} \gamma^{k-t} R_k) - V_{\psi}(s_t)$